# SQL & cache 방식 차감 성능 테스트  
- cache 이용한 차감 전체 차감 속도 측정(stream server 안에서)  
- snapshot 저장하지 않고, cache로 저장하고 이를 기반으로 사용할 경우 성능 테스트  
- 이 경우에도 주기적으로 snapshot 저장해야 함.  
- 중간 실패 후 재실행시, snapshot 에서 append request를 조합해 동일한 결과를 만들 수 있어야 사용 가능.  
- cache 사용 시, DAG 가 누적되는 현상에 유의해야 한다. 성능이 기하급수로 느려 짐.  


In [1]:
# 필요 라이브러리 임포트  
import socket
import sys
import os
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from os.path import abspath
# import findspark
import time 
import numpy as np 
import pandas as pd
import pyarrow 

In [2]:
# 환경변수 정의  
scale = 1000 # 1000 만 건 수준
PRJ_ROOT = '/user/root'
APP_NAME = 'RDD-Pandas'
DB_NAME = 'inven'

# 데이터의 파일 포맷 및 파일명  
tbl_setop_name = 'inven/table-set-6m-20-1000'
file_format = 'parquet'

In [3]:
# 스파크 생성 
def spark_creation():
    spark = SparkSession.builder.master('yarn').appName(APP_NAME)\
    .config('spark.rpc.message.maxSize', '1024')\
    .config('spark.sql.execution.arrow.enabled', 'true')\
    .config('spark.driver.cores', '1').config('spark.driver.memory', '7g')\
    .config('spark.num.executors', '3')\
    .config('spark.executor.cores', '1').config('spark.executor.memory', '7g')\
    .config('spark.jars', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar')\
    .config('spark.driver.extraClassPath', '/hive-bin/lib/mysql-connector-java-5.1.49-bin.jar').getOrCreate()
    #     .config('spark.sql.execution.arrow.enabled', 'true')\
    # spark.rpc.message.maxSize  240007497 
    sc = spark.sparkContext
    sc
    return spark

In [4]:
%%time
spark = spark_creation()
spark

/usr/local/lib/python3.6/dist-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


CPU times: user 16.8 ms, sys: 26.4 ms, total: 43.2 ms
Wall time: 20.4 s


# 최초 적재 & 캐싱 & 차감 캐싱  
최초 완전 적재 후에 이에 대해 연산 후, 캐싱한다.  
캐싱 결과를 빠르게 사용할 수 있는 지 확인하고, 캐싱 시간을 확인  
  


## Data 최초 적재 및 캐싱 
- 최초 select 100 : 25 초 
- 2번째 : filter : 1.1 초  
- 캐싱시 연산 스택이 누적되서, 일정 횟수 이상 지나면 2배씩 느려진다.  

In [5]:
# 샘플 데이터 형식 정의. 읽기/쓰기 편의 제공. 
def define_schema():
    from pyspark.sql.types import StructType, StructField, StringType, LongType, FloatType
    columns = [
        StructField("setop", StringType())
        , StructField("stype", StringType())
        , StructField("inv_rate_01", FloatType())
        , StructField("inv_val_01", LongType())
        , StructField("inv_req_01", LongType())
        , StructField("inv_rate_02", FloatType())
        , StructField("inv_val_02", LongType())
        , StructField("inv_req_02", LongType())
        , StructField("inv_rate_03", FloatType())
        , StructField("inv_val_03", LongType())
        , StructField("inv_req_03", LongType())
        , StructField("inv_rate_04", FloatType())
        , StructField("inv_val_04", LongType())
        , StructField("inv_req_04", LongType())
        , StructField("inv_rate_05", FloatType())
        , StructField("inv_val_05", LongType())
        , StructField("inv_req_05", LongType())        
        , StructField("inv_rate_06", FloatType())
        , StructField("inv_val_06", LongType())
        , StructField("inv_req_06", LongType())        
    ]
    sample_schema = StructType(columns)
    return sample_schema

In [20]:
%%time
## 인벤 기준 정보 조회  
# 최초 select 100 : 25 초 
# 2번째 : filter : 1.1 초  
tbl_name = f"setop_view_mem"
spark.read.format(file_format).load(tbl_setop_name).createOrReplaceTempView(tbl_name)
spark.catalog.cacheTable(tbl_name)
print(spark.catalog.isCached(tbl_name))
# 지연연산을 바로 호출 
spark.sql(f"select * from {tbl_name} where stype='ST_A' limit 5 ").show()

+------------+-----+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+
|       setop|stype|inv_rate_01|inv_val_01|inv_req_01|inv_rate_02|inv_val_02|inv_req_02|inv_rate_03|inv_val_03|inv_req_03|inv_rate_04|inv_val_04|inv_req_04|inv_rate_05|inv_val_05|inv_req_05|inv_rate_06|inv_val_06|inv_req_06|
+------------+-----+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+-----------+----------+----------+
|ST_A_0000000| ST_A|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|
|ST_A_0000001| ST_A|     1.0E-5|      1000|         0|     1.0E-5|      1000|         0|     1.0E-5|

In [21]:
# from pyspark.storagelevel import StorageLevel
# sdf = spark.sql(f"select * from {tbl_name} where stype='ST_A' limit 5 ")
# sdf.persist(StorageLevel.MEMORY_ONLY_2)
# sdf

In [24]:
# 차감 연산 + 캐싱  : 27 초  
# 차감하고 그 결과를 저장해 두고 다시 연산 기반으로 사용해야 하는 상황을 흉내내기 위해 차감 연산  
# 차감한 결과를 result 에 저장. 여기에서 리포트 생성. 
# result를 다시 setop_view 캐시에 덮어쓰고, 요청 들어올 때 여기에서 다시 차감 실행. 
def calc_caching(idx=0, stype='ST_A', amount=1):
    view_name = f"setop_view_mem"
    sql_minus = f"""
    SELECT * FROM {view_name} WHERE stype!='ST_J' 
    UNION ALL 
    SELECT 
    setop , stype 
    , inv_rate_01, (inv_val_01 - 3) inv_val_01, inv_req_01 
    , inv_rate_02, (inv_val_02 - 3) inv_val_02, inv_req_02 
    , inv_rate_03, (inv_val_03 - 3) inv_val_03, inv_req_03
    , inv_rate_04, (inv_val_04 - 3) inv_val_04, inv_req_04 
    , inv_rate_05, (inv_val_05 - 3) inv_val_05, inv_req_05 
    , inv_rate_06, (inv_val_06 - 3) inv_val_06, inv_req_06 
    FROM {view_name} WHERE stype='ST_J' 
    """
    spark.sql(sql_minus).createOrReplaceTempView("result")
    
    #spark.sql(sql_minus).createOrReplaceTempView("setop_view")
    #df_c = spark.sql(sql_minus).toDF()

    spark.catalog.clearCache()
    print(f"cached 1 : {spark.catalog.isCached(view_name)}")
    spark.catalog.uncacheTable(view_name)
    print(f"cached 2 : {spark.catalog.isCached(view_name)}")
    # 캐시 덮어쓰기  : 22 초 정도  
    view_next = f"setop_view_mem"
    spark.sql("select * from result").createOrReplaceTempView(view_next)
    df = spark.sql(f"select * from {view_next}")
    df.persist(StorageLevel.MEMORY_ONLY_2)
    spark.catalog.uncacheTable("result")
    print(f"result  : {spark.catalog.isCached('result')}")
    # print(spark.catalog.isCached('setop_view'))
    # spark.catalog.cacheTable("setop_view")
    # spark.catalog.isCached('setop_view')
    sql = f"select sum(inv_val_01), stype from {view_next} WHERE stype='ST_J' group by stype"
    # print(sql)
    spark.sql(sql).show()
    # print(df)

In [26]:
%%time
from timeit import default_timer as timer
for i in range(0, 5): 
    start = timer()
    calc_caching(i, 'ST_J', 100)
    end = timer()
    print(end - start)

cached 1 : False
cached 2 : False
result  : False
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

2.2328552060000675
cached 1 : False
cached 2 : False
result  : False
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

3.219343075999859
cached 1 : False
cached 2 : False
result  : False
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

4.826037558999815
cached 1 : False
cached 2 : False
result  : False
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

8.15038777399991
cached 1 : False
cached 2 : False
result  : False
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

14.313238274000014
CPU times: user 50.2 ms, sys: 15.7 ms, total: 65.9 ms


In [53]:
%%time
spark.sql("select sum(inv_val_01), stype from setop_view_40 group by stype").show(1)

+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      970000000| ST_J|
+---------------+-----+
only showing top 1 row

CPU times: user 3.71 ms, sys: 1.07 ms, total: 4.78 ms
Wall time: 4.65 s


In [71]:
spark.sql("select * from setop_view_mem").write.save(path="out_parq", format=file_format, mode="overwrite")

---  

In [90]:
%%time
# 2 차 테스트  

tbl_name = "setop_view_mem_00"
df = spark.read.format(file_format).load(tbl_setop_name)
df.persist(StorageLevel.MEMORY_ONLY_2)
# df.createOrReplaceTempView(tbl_name)
# print(df.isCached(tbl_name))

CPU times: user 1.5 ms, sys: 3.41 ms, total: 4.91 ms
Wall time: 1.85 s


DataFrame[setop: string, stype: string, inv_rate_01: float, inv_val_01: bigint, inv_req_01: bigint, inv_rate_02: float, inv_val_02: bigint, inv_req_02: bigint, inv_rate_03: float, inv_val_03: bigint, inv_req_03: bigint, inv_rate_04: float, inv_val_04: bigint, inv_req_04: bigint, inv_rate_05: float, inv_val_05: bigint, inv_req_05: bigint, inv_rate_06: float, inv_val_06: bigint, inv_req_06: bigint]

In [91]:
# 차감 연산 + 캐싱  : 27 초  
# 차감하고 그 결과를 저장해 두고 다시 연산 기반으로 사용해야 하는 상황을 흉내내기 위해 차감 연산  
# 차감한 결과를 result 에 저장. 여기에서 리포트 생성. 
# result를 다시 setop_view 캐시에 덮어쓰고, 요청 들어올 때 여기에서 다시 차감 실행. 


def calc_caching(df, idx=0, stype='ST_A', amount=1):
    view_name = f"setop_view_mem_{i:02}"
    view_name_next = f"setop_view_mem_{i+1:02}"
    sql_minus = f"""
    SELECT * FROM {view_name} WHERE stype!='ST_J' 
    UNION ALL 
    SELECT 
    setop , stype 
    , inv_rate_01, (inv_val_01 - 3) inv_val_01, inv_req_01 
    , inv_rate_02, (inv_val_02 - 3) inv_val_02, inv_req_02 
    , inv_rate_03, (inv_val_03 - 3) inv_val_03, inv_req_03
    , inv_rate_04, (inv_val_04 - 3) inv_val_04, inv_req_04 
    , inv_rate_05, (inv_val_05 - 3) inv_val_05, inv_req_05 
    , inv_rate_06, (inv_val_06 - 3) inv_val_06, inv_req_06 
    FROM {view_name} WHERE stype='ST_J' 
    """
    # spark.sql(sql_minus).createOrReplaceTempView("result")
    
    df_new = spark.sql(sql_minus)
    df_new.persist(StorageLevel.MEMORY_ONLY_2)
    spark.sparkContext.setCheckpointDir(".")
    df_new.checkpoint()
    df.unpersist()
    old_cached = spark.catalog.isCached(view_name)
    print(f"{view_name} cached un : {old_cached}")
    
    df_new.createOrReplaceTempView(view_name_next)
    sql = f"select sum(inv_val_01), stype from {view_name_next} WHERE stype='ST_J' group by stype"
    new_cached = spark.catalog.isCached(view_name_next)
    print(f"{view_name_next} cached un : {new_cached}")

    spark.sql(sql).show()
    return df_new
    # print(df)

In [92]:
%%time
df.createOrReplaceTempView("setop_view_mem_00")
spark.sql(" select sum(inv_val_01), stype from setop_view_mem_00 WHERE stype='ST_J' group by stype ").show()
print(f"cached 1 : {spark.catalog.isCached('setop_view_mem_00')}")

+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|     1000000000| ST_J|
+---------------+-----+

cached 1 : True
CPU times: user 9.41 ms, sys: 1.64 ms, total: 11 ms
Wall time: 22.4 s


In [52]:
spark.catalog.clearCache()
df.unpersist()

DataFrame[setop: string, stype: string, inv_rate_01: float, inv_val_01: bigint, inv_req_01: bigint, inv_rate_02: float, inv_val_02: bigint, inv_req_02: bigint, inv_rate_03: float, inv_val_03: bigint, inv_req_03: bigint, inv_rate_04: float, inv_val_04: bigint, inv_req_04: bigint, inv_rate_05: float, inv_val_05: bigint, inv_req_05: bigint, inv_rate_06: float, inv_val_06: bigint, inv_req_06: bigint]

In [93]:
%%time
from timeit import default_timer as timer
for i in range(0, 10): 
    start = timer()
    df_new = calc_caching(df, i, 'ST_J', 100)
    df = df_new
    end = timer()
    print(end - start)

setop_view_mem_00 cached un : False
setop_view_mem_01 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      997000000| ST_J|
+---------------+-----+

37.443113751000055
setop_view_mem_01 cached un : False
setop_view_mem_02 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      994000000| ST_J|
+---------------+-----+

35.804371866999645
setop_view_mem_02 cached un : False
setop_view_mem_03 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      991000000| ST_J|
+---------------+-----+

36.55269981799938
setop_view_mem_03 cached un : False
setop_view_mem_04 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      988000000| ST_J|
+---------------+-----+

42.36437893099992
setop_view_mem_04 cached un : False
setop_view_mem_05 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      98500000

---  
# no ckpt explicit save & load  


In [9]:
%%time
# 2 차 테스트  
from pyspark.storagelevel import StorageLevel
tbl_name = "setop_view_mem_00"
df = spark.read.format(file_format).load(tbl_setop_name)
df.persist(StorageLevel.MEMORY_ONLY_2)
# df.createOrReplaceTempView(tbl_name)
# print(df.isCached(tbl_name))

CPU times: user 3.23 ms, sys: 1.5 ms, total: 4.73 ms
Wall time: 498 ms


DataFrame[setop: string, stype: string, inv_rate_01: float, inv_val_01: bigint, inv_req_01: bigint, inv_rate_02: float, inv_val_02: bigint, inv_req_02: bigint, inv_rate_03: float, inv_val_03: bigint, inv_req_03: bigint, inv_rate_04: float, inv_val_04: bigint, inv_req_04: bigint, inv_rate_05: float, inv_val_05: bigint, inv_req_05: bigint, inv_rate_06: float, inv_val_06: bigint, inv_req_06: bigint]

In [30]:

def calc_caching(df, path, idx=0, stype='ST_A', amount=1):
    # df = spark.read.format("parquet").load("inven/ckpt")
    df.persist(StorageLevel.MEMORY_ONLY_2)
    
    view_name = f"setop_view_mem_{i:02}"
    view_name_next = f"setop_view_mem_{i+1:02}"
    df.createOrReplaceTempView(view_name)
    sql_minus = f"""
    SELECT * FROM {view_name} WHERE stype!='ST_J' 
    UNION ALL 
    SELECT 
    setop , stype 
    , inv_rate_01, (inv_val_01 - 3) inv_val_01, inv_req_01 
    , inv_rate_02, (inv_val_02 - 3) inv_val_02, inv_req_02 
    , inv_rate_03, (inv_val_03 - 3) inv_val_03, inv_req_03
    , inv_rate_04, (inv_val_04 - 3) inv_val_04, inv_req_04 
    , inv_rate_05, (inv_val_05 - 3) inv_val_05, inv_req_05 
    , inv_rate_06, (inv_val_06 - 3) inv_val_06, inv_req_06 
    FROM {view_name} WHERE stype='ST_J' 
    """
    # spark.sql(sql_minus).createOrReplaceTempView("result")
    
    df_new = spark.sql(sql_minus)
    df_new.persist(StorageLevel.MEMORY_ONLY_2)

    #df_new.unpersist()
    #df_new.write.save(path="inven/ckpt2", format="parquet", mode="overwrite")
    
    # 차감결과 출력 
    df_new.createOrReplaceTempView(view_name_next)
    sql = f"select sum(inv_val_01), stype from {view_name_next} WHERE stype='ST_J' group by stype"
    new_cached = spark.catalog.isCached(view_name_next)
    print(f"{view_name_next} cached un : {new_cached}")
    spark.sql(sql).show()
    
    return df_new
    # print(df)

In [18]:
%%time
# df.createOrReplaceTempView("setop_view_mem_00")
# spark.sql(" select sum(inv_val_01), stype from setop_view_mem_00 WHERE stype='ST_J' group by stype ").show()
# print(f"cached 1 : {spark.catalog.isCached('setop_view_mem_00')}")

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.68 µs


In [31]:
%%time
# 1 회 실행시 평균 36 초 정도...  
# ckpt와 결합해서 n회 까지 ckpt & persist로 조금 더 빠르게 처리하고, 
# n회 되면, 직접 파일에 쓰고, 다시 읽어서 lineage 초기화 시킨다.  
from timeit import default_timer as timer
from pyspark.storagelevel import StorageLevel

path = tbl_setop_name
for i in range(0, 10): 
    start = timer()
    df = spark.read.format("parquet").load(path)
    df_new = calc_caching(df, i, 'ST_J', 100)
    path = f"inven/ckpt_{i:02}"
    df_new.write.save(path=path, format="parquet", mode="overwrite")
    df = df_new
    end = timer()
    print(end - start)

setop_view_mem_01 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      997000000| ST_J|
+---------------+-----+

25.322488365000027
setop_view_mem_02 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      994000000| ST_J|
+---------------+-----+

36.7830876070002
setop_view_mem_03 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      991000000| ST_J|
+---------------+-----+

36.53329946599979
setop_view_mem_04 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      988000000| ST_J|
+---------------+-----+

37.34806484899991
setop_view_mem_05 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      985000000| ST_J|
+---------------+-----+

35.57926315700024
setop_view_mem_06 cached un : True
+---------------+-----+
|sum(inv_val_01)|stype|
+---------------+-----+
|      982000000| ST_J|

In [32]:
spark.stop()